In [2]:
import cv2
import time
#from dotenv import load_dotenv
import os

In [ ]:
def capture_rtsp_stream(rtsp_url):
    cap = cv2.VideoCapture(rtsp_url)

    if not cap.isOpened():
        print("Ошибка: Не удалось открыть видеопоток.")
        return

    frame_counter = 0
    start_time = time.time()
    fps_display_interval = 1.0
    last_fps_update = time.time()

    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Ошибка: Не удалось получить кадр из видеопотока.")
            break
        frame_counter += 1

        current_time = time.time()
        elapsed_time = current_time - start_time
        if elapsed_time >= fps_display_interval:
            fps = frame_counter / elapsed_time
            frame_counter = 0
            start_time = current_time

            last_fps_update = current_time

        text = f"FPS: {fps:.2f}" if 'fps' in locals() else "FPS: N/A"
        (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        cv2.rectangle(frame, (10, 10), (10 + text_width, 10 + text_height + 10), (0, 0, 0), -1)  # Закрашиваем фон
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("RTSP Stream", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
# load_dotenv()
# rtsp_url = os.getenv("RTSP_URL")
rtsp_url = "rtsp://192.168.1.102:554/live/ch00_1"

if not rtsp_url:
    print("Ошибка: Переменная окружения RTSP_URL не установлена в файле .env.")
else:
    capture_rtsp_stream(rtsp_url)

(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 

In [9]:
from onvif import ONVIFCamera

# Настройки камеры
CAMERA_IP = '192.168.1.102'  # Замените на IP вашей камеры
CAMERA_PORT = 554            # Порт ONVIF (обычно 80 или 8080)
CAMERA_USER = 'admin'        # Логин
CAMERA_PASSWORD = 'admin' # Пароль

#rtsp://192.168.1.102/ptz?direction=up

In [10]:
# Создаем объект камеры
camera = ONVIFCamera(CAMERA_IP, CAMERA_PORT, CAMERA_USER, CAMERA_PASSWORD)

KeyboardInterrupt: 

In [ ]:


# Инициализация сервиса PTZ
ptz_service = camera.create_ptz_service()
media_service = camera.create_media_service()

# Получаем профиль камеры (необходим для PTZ-команд)
profiles = media_service.GetProfiles()
profile_token = profiles[0].token

# Функция для перемещения камеры
def move_camera(pan, tilt, zoom=0):
    """
    Перемещает камеру в заданном направлении.
    :param pan: Горизонтальное перемещение (-1.0 до 1.0)
    :param tilt: Вертикальное перемещение (-1.0 до 1.0)
    :param zoom: Масштабирование (0.0 до 1.0)
    """
    request = ptz_service.create_type('ContinuousMove')
    request.ProfileToken = profile_token
    request.Velocity = {
        'PanTilt': {'x': pan, 'y': tilt},
        'Zoom': {'x': zoom}
    }
    ptz_service.ContinuousMove(request)

# Примеры команд для перемещения камеры
if __name__ == "__main__":
    try:
        print("Перемещение камеры вверх...")
        move_camera(pan=0, tilt=0.5)  # Движение вверх

        print("Ожидание 2 секунд...")
        import time
        time.sleep(2)

        print("Перемещение камеры вправо...")
        move_camera(pan=0.5, tilt=0)  # Движение вправо

        time.sleep(2)

        print("Остановка движения...")
        ptz_service.Stop({'ProfileToken': profile_token})  # Остановка движения

    except Exception as e:
        print(f"Произошла ошибка: {e}")